In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.2 MB/s eta 0:00:00:00:0100:01


In [5]:
import sqlite3
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import torch

2025-07-20 17:23:25.552887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753032205.852931      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753032205.943146      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Step 1: Setup SQLite database
def setup_db():
    conn = sqlite3.connect("example.db")
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS employees")
    cursor.execute("""
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            salary INTEGER
        )
    """)
    cursor.executemany("INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)", [
        ("Alice", "Engineering", 120000),
        ("Bob", "Marketing", 80000),
        ("Charlie", "HR", 70000),
        ("David", "Engineering", 130000),
    ])
    conn.commit()
    conn.close()

setup_db()

# Step 2: Extract categorical values
def build_vector_index():
    conn = sqlite3.connect("example.db")
    cursor = conn.cursor()
    categorical_columns = ["name", "department"]
    data = []
    labels = []

    for col in categorical_columns:
        cursor.execute(f"SELECT DISTINCT {col} FROM employees")
        for val in cursor.fetchall():
            if val[0]:
                text = f"{col}: {val[0]}"
                data.append(text)
                labels.append(text)
    conn.close()

    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(data, convert_to_numpy=True)

    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, model, labels

In [7]:
index, embedder, label_lookup = build_vector_index()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# Step 3: Load Flan-T5 Small
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def generate_sql(question, context):
    prompt = f"""
You are a helpful assistant that writes SQL queries for a SQLite database.
The database has a table called 'employees' with columns: id, name, department, salary.

Context:
{context}

Question: {question}
Write the SQL query:
"""
    inputs = tokenizer(prompt.strip(), return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 4: Run chatbot loop
def ask_bot(question):
    q_embed = embedder.encode([question])
    D, I = index.search(q_embed, k=3)
    context = "\n".join([label_lookup[i] for i in I[0]])

    sql = generate_sql(question, context)
    print("\nGenerated SQL:\n", sql)

    try:
        conn = sqlite3.connect("example.db")
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        print("\nResult:\n", rows)
    except Exception as e:
        print("\nSQL Error:", e)

In [ ]:
# Step 5: CLI loop
print("Ask about the employees DB (type 'exit' to quit):\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    ask_bot(user_input)

Ask about the employees DB (type 'exit' to quit):



You:  Show all employees in Engineering


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Generated SQL:
 id, name, department, salary

SQL Error: near "id": syntax error
